In [1]:
import torch

from detectron2 import model_zoo
from detectron2.modeling import build_model
from detectron2.engine import DefaultPredictor, DefaultTrainer
from detectron2.data.datasets import register_coco_instances
from detectron2.data import DatasetCatalog, MetadataCatalog
from detectron2.utils.visualizer import Visualizer
from detectron2.utils.visualizer import ColorMode
from detectron2.config import get_cfg
import detectron2.config
from detectron2.evaluation import COCOEvaluator, inference_on_dataset
from detectron2.data import build_detection_test_loader

# import some common libraries
import numpy as np
import os, json, cv2, random
import matplotlib.pyplot as plt
import csv

## Load the dataset


In [2]:
if torch.cuda.is_available():
    print(f'CUDAs available: {torch.cuda.device_count()}')
    torch.cuda.set_device(1) #Set default GPU here
else:
    print("CUDA is not available")

CUDAs available: 3


In [3]:
# Hyperparameters
# UPDATE THESE
checkpoint_name = "model_0009999.pth"
model_dir = 'output_models/Rooftop Identification Model'
backbone = 101
num_classes = 19

# Dataset to predict on
# UPDATE THESE
satImageDir = '../Satellite Image Retrieval/Satellite Images'
satAnnDir = '../Satellite Image Retrieval/satellite_images_coco.json'

# Output directory
output_dir = './model_output_final'
os.makedirs(output_dir, exist_ok=True)

print(f'Output Directory: {output_dir}')

Output Directory: ./model_output_final


In [6]:
# Satellite images to predict
register_coco_instances("satellite_images", {}, satAnnDir, satImageDir)
dataset = DatasetCatalog.get("satellite_images")

metadata=MetadataCatalog.get("satellite_images")

In [7]:
cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file(f"COCO-InstanceSegmentation/mask_rcnn_R_{backbone}_FPN_3x.yaml"))
cfg.DATASETS.TEST = ("satellite_images")
cfg.MODEL.ROI_HEADS.NUM_CLASSES = num_classes   
cfg.OUTPUT_DIR = output_dir

# Load the weights from training
cfg.MODEL.WEIGHTS = os.path.join(model_dir, checkpoint_name)  # path to the model we just trained
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5   # set a custom testing threshold
cfg.MODEL.RPN.NMS_THRESH = 0.5 # NMS threshold used on RPN proposals (default is 0.7)

predictor = DefaultPredictor(cfg)

### Perform model prediction

In [12]:
evaluator = COCOEvaluator("satellite_images", output_dir=cfg.OUTPUT_DIR)
val_loader = build_detection_test_loader(cfg, "satellite_images")
metric_dict = inference_on_dataset(predictor.model, val_loader, evaluator)

Loading and preparing results...
DONE (t=0.43s)
creating index...
index created!
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = -1.000
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = -1.000
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = -1.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = -1.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = -1.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = -1.000
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = -1.000
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = -1.000
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = -1.000
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = -1.000
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = -1.000
 Average Recall     (AR) @[ 